In [1]:
import simpy
import random
import numpy as np

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

In [3]:
router_customer = []
opr_1_break_times = []
opr_2_break_times = []
shift_started = True
time_operator_1 = False
time_operator_2 = False

In [4]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
        
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [5]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        # Voice recognition and routing
        router_customer.append(self.name)
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router_customer.remove(self.name)
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request() as req:
        
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                return 
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done at %g' % (self.name, self.env.now))
        
       

In [6]:
def is_available(opr):
    return opr.count == 0

In [7]:
#Determining breaks
def break_number():
    return np.random.poisson(8, size=2)

In [8]:
def insertionSort(operator_list):
    for i in range(1,len(operator_list)):
        val = operator_list[i]
        pos = i
        while pos>0 and operator_list[pos-1]>val:
            operator_list[pos]=operator_list[pos-1]
            pos = pos-1
        operator_list[pos]=val
    print(operator_list)

In [9]:
#Break request times and sorting
def break_time(opr_1_break, opr_2_break):
    for k in range(opr_1_break):
        opr_1_break_times.append(random.uniform(0,480))
    for k in range(opr_2_break):
        opr_2_break_times.append(random.uniform(0,480))
    insertionSort(opr_1_break_times)
    insertionSort(opr_2_break_times)

In [10]:
def shift(env):
    global shift_started
    wait = env.timeout(480)
    yield wait
    shift_started = True

In [11]:
def customer_generator(env):
    global shift_started
    opr_1_break = 0
    opr_2_break = 0
    j = 0
    breaks_1 = 0
    breaks_2 = 0
    for i in range(1000):
        #To determine whether the shift is started, a process is opened.
        #Every 480 a new shift starts.
        if(shift_started):
            [opr_1_break, opr_2_break] = break_number()
            print(opr_1_break)
            print(opr_2_break)
            break_time(opr_1_break, opr_2_break)
            breaks_1 = 0
            breaks_2 = 0
            j += 1
            shift_started = False
            env.process(shift(env))
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if len(router_customer) < 100:
            customer = Customer('Customer %s' %(i+1), env)
        

In [12]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env))
env.run() 

[44.63307524333047, 74.39519318760645, 209.12059240981336, 255.9664319411964, 292.4094909717117, 385.08427703724, 387.8161389706391, 397.1018255104171, 474.21742412177036]
[34.07604646614267, 90.0478027519914, 126.55138170720205, 174.5488473525917, 226.6161423659573, 393.61533834425836, 414.16006451794135, 434.4019150966053]
Customer 1 initiated a call at 7.82907
Customer 1 is rooted to operator 2 at 14.8337
Customer 1 is assigned to the operator 2 at 14.8337
Customer 1 is done at 18.1629
Customer 2 initiated a call at 29.7356
Customer 3 initiated a call at 31.2913
Customer 4 initiated a call at 31.5172
Customer 4 is rooted to operator 2 at 33.3233
Customer 4 is assigned to the operator 2 at 33.3233
Customer 2 hangs up the call at 33.4232
Customer 5 initiated a call at 34.5759
Customer 5 is rooted to operator 2 at 35.1087
Customer 3 is rooted to operator 2 at 37.8012
Customer 4 is done at 38.9678
Customer 5 is assigned to the operator 2 at 38.9678
Customer 6 initiated a call at 39.3183

Customer 497 initiated a call at 3081.46
Customer 497 is rooted to operator 2 at 3081.63
Customer 497 is assigned to the operator 2 at 3081.63
Customer 498 initiated a call at 3084.73
Customer 498 is rooted to operator 1 at 3084.78
Customer 498 is assigned to the operator 1 at 3084.78
Customer 497 is done at 3085
Customer 499 initiated a call at 3088.83
Customer 498 is done at 3096.76
Customer 500 initiated a call at 3097.57
Customer 500 hangs up the call at 3098.88
Customer 501 initiated a call at 3100.47
Customer 502 initiated a call at 3100.97
Customer 503 initiated a call at 3102.31
Customer 501 is rooted to operator 2 at 3103.47
Customer 501 is assigned to the operator 2 at 3103.47
Customer 503 is rooted to operator 2 at 3104.48
Customer 504 initiated a call at 3104.78
Customer 501 is done at 3105.32
Customer 503 is assigned to the operator 2 at 3105.32
Customer 499 is rooted to operator 1 at 3105.62
Customer 499 is assigned to the operator 1 at 3105.62
Customer 505 initiated a ca

Customer 679 is rooted to operator 1 at 4182.61
Customer 679 is assigned to the operator 1 at 4182.61
Customer 680 is rooted to operator 2 at 4183.59
Customer 680 is assigned to the operator 2 at 4183.59
Customer 681 initiated a call at 4188.05
Customer 680 is done at 4188.41
Customer 682 initiated a call at 4192.79
Customer 681 is rooted to operator 2 at 4193.06
Customer 681 is assigned to the operator 2 at 4193.06
Customer 679 is done at 4193.43
Customer 683 initiated a call at 4193.81
Customer 683 is rooted to operator 2 at 4194.45
Customer 682 is rooted to operator 2 at 4195.84
Customer 684 initiated a call at 4199.9
Customer 681 is done at 4199.91
Customer 683 is assigned to the operator 2 at 4199.91
Customer 684 is rooted to operator 1 at 4202.91
Customer 684 is assigned to the operator 1 at 4202.91
Customer 682 is left the operator 2 after 10 minutes at 4205.84
Customer 683 is done at 4206.68
Customer 684 is done at 4211.14
Customer 685 initiated a call at 4213.98
Customer 685 i